<a href="https://colab.research.google.com/github/matdjohnson-at-umass-dot-edu/cs646-final-project/blob/main/CS646_Final_Project_Preprocessing1_Instance_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
! pip install transformers

In [ ]:
from datasets import load_dataset, concatenate_datasets
from tqdm import tqdm
from google.colab import drive
from datasets import load_from_disk
from datasets import Dataset
import os
import torch
import gc
import time
from threading import Lock
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, AutoModel
import logging
os.environ["TOKENIZERS_PARALLELISM"] = "false"

drive.mount('/content/drive')

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_in_qrel') \
    or not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_not_in_qrel') \
    or not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_in_qrel') \
    or not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_not_in_qrel'):

    qrels = load_dataset('BeIR/msmarco-qrels')

    print(qrels)

    train_corpus_ids = list()
    train_queries_ids = list()
    for entry in qrels['train']:
        train_corpus_ids.append(entry['corpus-id'])
        train_queries_ids.append(entry['query-id'])
    train_queries_ids = list(set(train_queries_ids))
    train_corpus_ids = list(set(train_corpus_ids))

    validation_corpus_ids = list()
    validation_queries_ids = list()
    for entry in qrels['validation']:
        validation_corpus_ids.append(entry['corpus-id'])
        validation_queries_ids.append(entry['query-id'])
    validation_corpus_ids = list(set(validation_corpus_ids))
    validation_queries_ids = list(set(validation_queries_ids))

    test_corpus_ids = list()
    test_queries_ids = list()
    for entry in qrels['test']:
        test_corpus_ids.append(entry['corpus-id'])
        test_queries_ids.append(entry['query-id'])
    test_queries_ids = list(set(test_queries_ids))
    test_corpus_ids = list(set(test_corpus_ids))

    corpus_ids = set(train_corpus_ids + validation_corpus_ids + test_corpus_ids)
    queries_ids = set(train_queries_ids + validation_queries_ids + test_queries_ids)

    print(f"len(corpus_ids):{len(corpus_ids)} len(train_corpus_ids):{len(train_corpus_ids)} len(validation_corpus_ids):{len(validation_corpus_ids)} len(test_corpus_ids):{len(test_corpus_ids)}")
    print(f"len(queries_ids):{len(queries_ids)} len(train_queries_ids):{len(train_queries_ids)} len(validation_queries_ids):{len(validation_queries_ids)} len(test_queries_ids):{len(test_queries_ids)}")

    ms_marco_corpus = load_dataset('BeIR/msmarco', 'corpus')
    ms_marco_queries = load_dataset('BeIR/msmarco', 'queries')

    print(ms_marco_corpus)

    ms_marco_corpus_in_qrel = ms_marco_corpus['corpus'].filter(lambda example: int(example['_id']) in corpus_ids)
    ms_marco_corpus_not_in_qrel = ms_marco_corpus['corpus'].filter(lambda example: int(example['_id']) not in corpus_ids)

    ms_marco_queries_in_qrel = ms_marco_queries['queries'].filter(lambda example: int(example['_id']) in queries_ids)
    ms_marco_queries_not_in_qrel = ms_marco_queries['queries'].filter(lambda example: int(example['_id']) not in queries_ids)

    print(ms_marco_corpus_in_qrel)
    print(ms_marco_corpus_not_in_qrel)
    print(ms_marco_queries_in_qrel)
    print(ms_marco_queries_not_in_qrel)

    if not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_in_qrel'):
        os.makedirs('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_in_qrel')
    ms_marco_corpus_in_qrel.save_to_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_in_qrel', max_shard_size='250MB')
    if not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_not_in_qrel'):
        os.makedirs('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_not_in_qrel')
    ms_marco_corpus_not_in_qrel.save_to_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_not_in_qrel', max_shard_size='250MB')
    if not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_in_qrel'):
        os.makedirs('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_in_qrel')
    ms_marco_queries_in_qrel.save_to_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_in_qrel', max_shard_size='250MB')
    if not os.path.exists('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_not_in_qrel'):
        os.makedirs('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_not_in_qrel')
    ms_marco_queries_not_in_qrel.save_to_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_not_in_qrel', max_shard_size='250MB')

else:
    ms_marco_corpus_in_qrel = load_from_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_in_qrel')
    ms_marco_corpus_in_qrel = ms_marco_corpus_in_qrel.remove_columns(['tokenization', 'embedding'])
    ms_marco_corpus_not_in_qrel = load_from_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_not_in_qrel')
    ms_marco_corpus_not_in_qrel = ms_marco_corpus_not_in_qrel.remove_columns(['tokenization', 'embedding'])
    ms_marco_queries_in_qrel = load_from_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_in_qrel')
    ms_marco_queries_in_qrel = ms_marco_queries_in_qrel.remove_columns(['tokenization', 'embedding'])
    ms_marco_queries_not_in_qrel = load_from_disk('/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_not_in_qrel')
    ms_marco_queries_not_in_qrel = ms_marco_queries_not_in_qrel.remove_columns(['tokenization', 'embedding'])



In [ ]:
model_name = 'princeton-nlp/sup-simcse-roberta-large'
max_model_pool_size = 25
dataset_start_ind = 937500
dataset_end_ind = 1000000
log_frequency = 10

class ModelHolder:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

def populate_model_pool(model_pool=list(), pool_size=7):
    for i in range(0, pool_size):
        model_pool.append(
            ModelHolder(
                AutoModel.from_pretrained(model_name).cuda(),
                AutoTokenizer.from_pretrained(model_name)
            )
        )
    return model_pool

def process_batch(batch_num, log_dir, tokenizer, model, batch):
    tokenizations = list()
    embeddings = list()
    for i in range(0, len(batch)):
        tokenization = tokenizer(batch['text'][i], padding=True, truncation=True, return_tensors='pt').to('cuda')
        model_output = model(**tokenization, return_dict=True)
        embeddings.append(model_output.last_hidden_state.detach().to('cpu').numpy().flatten())
        tokenizations.append(tokenization['input_ids'].detach().to('cpu').numpy().flatten())
        del model_output
        del tokenization
        torch.cuda.empty_cache()
        gc.collect()
        if (i+1) % log_frequency == 0 or (i+1) == 1:
            timestamp = time.strftime("%Y-%m-%dT%H:%M:%S", time.localtime())
            log_file = open(f"{log_dir}/batch-{dataset_start_ind}-{dataset_end_ind}.log", "a")
            log_file.write(f"{timestamp}: batch-runner-{batch_num} - completed {i+1} of {len(batch)}\n")
            log_file.close()
    return batch, tokenizations, embeddings

model_pool = populate_model_pool(pool_size=max_model_pool_size)
executor = ThreadPoolExecutor(max_workers=max_model_pool_size)

In [ ]:
# corpus_dir_for_batch = f"/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_in_qrel_embs_{dataset_start_ind}-{dataset_end_ind}"
corpus_dir_for_batch = f"/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_corpus_not_in_qrel_embs_{dataset_start_ind}-{dataset_end_ind}"
# corpus_dir_for_batch = f"/content/drive/MyDrive/CS646-FinalProject/datasets/ms_marco_queries_in_qrel_embs_{dataset_start_ind}-{dataset_end_ind}"

if not os.path.exists(corpus_dir_for_batch):
    os.makedirs(corpus_dir_for_batch)

futures_list = list()

# ms_marco_corpus_selection = ms_marco_corpus_in_qrel.select(range(dataset_start_ind, dataset_end_ind))
ms_marco_corpus_selection = ms_marco_corpus_not_in_qrel.select(range(dataset_start_ind, dataset_end_ind))
# ms_marco_corpus_selection = ms_marco_queries_in_qrel.select(range(dataset_start_ind, dataset_end_ind))

torch.cuda.empty_cache()
gc.collect()
for i in range(0, max_model_pool_size):
    batch_size = (dataset_end_ind - dataset_start_ind) // max_model_pool_size
    batch_lower_bound = i * batch_size
    if i == max_model_pool_size - 1:
        batch_upper_bound = batch_upper_bound = len(ms_marco_corpus_selection)
    else:
        batch_upper_bound = (i+1) * batch_size
    batch = ms_marco_corpus_selection.select(range(batch_lower_bound, batch_upper_bound))
    future = executor.submit(process_batch, i, corpus_dir_for_batch, model_pool[i].tokenizer, model_pool[i].model, batch)
    futures_list.append(future)

ms_marco_corpus_selection_preprocessed_list = list()
while len(futures_list) > 0:
    for i in range(0, len(futures_list)):
        future = futures_list[i]
        if future.done():
            batch, tokenizations, embeddings = future.result()
            batch = batch.add_column('tokenization', tokenizations)
            batch = batch.add_column('embedding', embeddings)
            ms_marco_corpus_selection_preprocessed_list.append(batch)
            futures_list.pop(i)
            break

for i in range(0, len(ms_marco_corpus_selection_preprocessed_list)):
    ms_marco_corpus_selection_preprocessed_list[i].to_parquet(corpus_dir_for_batch + f"/subset_{i}.parquet")

ms_marco_corpus_selection_preprocessed = concatenate_datasets(ms_marco_corpus_selection_preprocessed_list)
print(ms_marco_corpus_selection_preprocessed)
for i in range(10):
    print(f"{ms_marco_corpus_selection_preprocessed[i]}"[:1500])